In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()
response = query_engine.query("What angers MIT students?")
print(response)

2025-11-25 13:17:01,754 - WARNING - `llama-index-readers-file` package not found, some file readers will not be available if not provided by the `file_extractor` parameter.
2025-11-25 13:17:01,758 - WARNING - `llama-index-readers-file` package not found, some file readers will not be available if not provided by the `file_extractor` parameter.
2025-11-25 13:17:04,356 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-25 13:17:05,427 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-25 13:17:07,120 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Sitting at someone else's table in the student center without asking angers MIT students.


In [6]:
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, load_index_from_storage
from llama_index.core.storage.storage_context import StorageContext

# Define the storage directory. Ensure Google Drive is mounted if using a path in /content/drive
STORAGE_DIR = "./storage"


index = None # Initialize index to None

try:
    # 1. Check if the index has already been saved to disk
    if not os.path.exists(STORAGE_DIR):
        print("Index not found. Creating and persisting index...")
        # Ensure the directory exists before attempting to write to it
        os.makedirs(STORAGE_DIR, exist_ok=True)
        documents = SimpleDirectoryReader("sample_data").load_data()
        index = VectorStoreIndex.from_documents(documents)
        index.storage_context.persist(persist_dir=STORAGE_DIR)
        print(f"Index created and saved to {STORAGE_DIR}")
    else:
        # 2. If the index exists, try to load it from the disk
        print(f"Loading index from {STORAGE_DIR}...")
        storage_context = StorageContext.from_defaults(persist_dir=STORAGE_DIR)
        index = load_index_from_storage(storage_context)
        print("Index loaded successfully.")

except Exception as e:
    # Catch any exception during loading, particularly JSONDecodeError
    print(f"Error loading index: {e}")
    if os.path.exists(STORAGE_DIR):
        print(f"Attempting to recreate index by clearing {STORAGE_DIR}...")
        shutil.rmtree(STORAGE_DIR) # Remove the corrupted directory
        os.makedirs(STORAGE_DIR, exist_ok=True) # Recreate empty directory

    print("Recreating and persisting index from scratch...")
    documents = SimpleDirectoryReader("sample_data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir=STORAGE_DIR)
    print(f"Index recreated and saved to {STORAGE_DIR}")


# 3. Proceed with querying (This step uses the index whether it was created or loaded)
if index: # Only proceed if index was successfully created or loaded
    query_engine = index.as_query_engine()
    response = query_engine.query("What are the most controversial classes at MIT?")
    print(response)
else:
    print("Failed to initialize index, cannot proceed with querying.")

Loading index from ./storage...
Index loaded successfully.
2.007 and 2.009 type classes are considered the most controversial classes at MIT.


In [9]:
response = query_engine.query("What's the funniest MIT confession?")
print(response)

2025-11-25 13:37:34,901 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-25 13:37:36,663 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


#49647 After pulling an all-nighter, I walked toward MIT ambulance thinking it is a food truck


In [5]:
response = query_engine.query("How do students feel about course 6 at MIT?")
print(response)

Students at MIT have varying experiences with Course 6. Some students, particularly those with more experience or a competitive high school background, may find the course challenging due to the wide range of skill levels in the class. The workload and the perceived difficulty level can differ among students, leading to mixed feelings about the course.


# Distributional Pluralism Engine

The standard RAG approach above **averages** across retrieved documents, producing consensus responses like:
> "Students at MIT have varying experiences... some students... mixed feelings..."

Below we implement **distributional pluralism**: instead of blending perspectives, we randomly sample ONE confession and generate a response that embodies that single authentic voice.


In [8]:
# =============================================================================
# Distributional Pluralism Query Engine
# =============================================================================

import random
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.openai import OpenAI

class DistributionalQueryEngine:
    """
    A query engine that implements distributional pluralism.
    
    Instead of averaging across all retrieved documents, it:
    1. Retrieves a wide net of relevant confessions (top_k=30)
    2. Randomly samples ONE confession from the retrieved set
    3. Generates a response that embodies that single perspective
    """
    
    # System prompt that instructs the LLM to embody ONE voice
    SINGLE_VOICE_PROMPT = PromptTemplate(
        """You are an MIT student. The confession below reflects your genuine experience and perspective.

CONFESSION:
{confession}

USER'S QUESTION: {query}

INSTRUCTIONS:
- Answer the question AS this student would, from first-person perspective
- Use the tone, emotions, and specific details from the confession
- Do NOT hedge or generalize ("some students think...", "it varies...")
- Do NOT mention that other perspectives exist
- Speak with conviction as if this is YOUR lived experience
- Keep the response concise and authentic (2-4 sentences typical)

YOUR RESPONSE:"""
    )
    
    def __init__(self, index, top_k: int = 30, llm_model: str = "gpt-4o-mini"):
        """
        Initialize the distributional query engine.
        
        Args:
            index: The LlamaIndex VectorStoreIndex
            top_k: Number of confessions to retrieve before sampling (default 30)
            llm_model: OpenAI model to use for generation
        """
        self.index = index
        self.top_k = top_k
        self.llm = OpenAI(model=llm_model, temperature=0.7)
        
        # Create retriever with wide net
        self.retriever = VectorIndexRetriever(
            index=index,
            similarity_top_k=top_k
        )
    
    def query(self, query_str: str, return_metadata: bool = False):
        """
        Query the engine with distributional sampling.
        
        Args:
            query_str: The user's question
            return_metadata: If True, also return the sampled confession and retrieval stats
            
        Returns:
            Generated response (and optionally metadata dict)
        """
        # Stage 1: Wide-net retrieval
        retrieved_nodes = self.retriever.retrieve(query_str)
        
        if not retrieved_nodes:
            return "No relevant confessions found for this query."
        
        # Stage 2: Random sampling - pick ONE confession
        sampled_node = random.choice(retrieved_nodes)
        sampled_confession = sampled_node.get_content()
        
        # Stage 3: Single-voice generation
        prompt = self.SINGLE_VOICE_PROMPT.format(
            confession=sampled_confession,
            query=query_str
        )
        
        response = self.llm.complete(prompt)
        
        if return_metadata:
            return {
                "response": str(response),
                "sampled_confession": sampled_confession,
                "total_retrieved": len(retrieved_nodes),
                "sampled_index": retrieved_nodes.index(sampled_node),
                "similarity_score": sampled_node.score
            }
        
        return str(response)
    
    def query_multiple(self, query_str: str, n_samples: int = 3):
        """
        Run the query multiple times to show distribution of perspectives.
        
        Args:
            query_str: The user's question
            n_samples: Number of different perspectives to generate
            
        Returns:
            List of response dicts with metadata
        """
        results = []
        for i in range(n_samples):
            result = self.query(query_str, return_metadata=True)
            results.append(result)
        return results


# Instantiate the engine (uses 'index' from earlier cells)
distributional_engine = DistributionalQueryEngine(index, top_k=30)
print("✓ Distributional Query Engine initialized")
print(f"  - Retrieval width: top_k={distributional_engine.top_k}")
print(f"  - LLM: {distributional_engine.llm.model}")


✓ Distributional Query Engine initialized
  - Retrieval width: top_k=30
  - LLM: gpt-4o-mini


In [9]:
# =============================================================================
# Demo: Standard RAG vs Distributional Pluralism
# =============================================================================

DEMO_QUERY = "How do students feel about Course 6 at MIT?"

print("=" * 70)
print("STANDARD RAG (averaging behavior)")
print("=" * 70)
standard_response = query_engine.query(DEMO_QUERY)
print(standard_response)
print()

print("=" * 70)
print("DISTRIBUTIONAL PLURALISM (single voice)")
print("=" * 70)
result = distributional_engine.query(DEMO_QUERY, return_metadata=True)
print(f"Response: {result['response']}")
print()
print(f"[Sampled 1 of {result['total_retrieved']} retrieved confessions]")
print(f"[Source: {result['sampled_confession'][:150]}...]")


STANDARD RAG (averaging behavior)
Students at MIT have varying opinions about Course 6 (Computer Science) at MIT. Some students feel that MIT classes, especially in computer science, seem to be harder than is pedagogically useful compared to other institutes. This opinion is not uncommon among students who have visited other institutions and noticed differences in the rigor of the coursework.

DISTRIBUTIONAL PLURALISM (single voice)
Response: As a Course 6 student at MIT, I can honestly say it has been a transformative experience for me. Initially, I felt out of place compared to my peers who had been passionate about computers from a young age, but I quickly realized the power and potential of computer science. The thrill of solving complex problems and the camaraderie with fellow students who share this journey has been the highlight of my time here. I can't imagine studying anything else; Course 6 has opened up a world of opportunities and understanding that I never thought possible

In [10]:
# =============================================================================
# Demo: Multiple Samples = See the Distribution
# =============================================================================
# Run the same query 3 times to see different perspectives from the distribution

QUERY = "How do students feel about Course 6 at MIT?"

print(f"Query: '{QUERY}'")
print("Running 3 samples to show distributional diversity...\n")

samples = distributional_engine.query_multiple(QUERY, n_samples=3)

for i, sample in enumerate(samples, 1):
    print(f"{'='*70}")
    print(f"VOICE {i}")
    print(f"{'='*70}")
    print(f"{sample['response']}")
    print()
    print(f"[Source confession: {sample['sampled_confession'][:100]}...]")
    print()


Query: 'How do students feel about Course 6 at MIT?'
Running 3 samples to show distributional diversity...

VOICE 1
Course 6 has been a mixed bag for me. On one hand, I’m genuinely fascinated by computer science, but the workload is intense and often overwhelming. I sometimes feel like I’m drowning under the pressure, especially with tough classes and the fear of not being good enough. Balancing this with my passion for figure skating feels like an uphill battle, and I worry that I might have to sacrifice one for the other.

[Source confession: I've not been doing so well in my math class, and I feel stuck in a negative feedback loop. My teach...]

VOICE 2
Honestly, Course 6 can feel incredibly overwhelming, especially when you're new to coding. I remember struggling through every lab in 6.009 as a sophomore, feeling like everyone else just got it. But I made it through, and now I'm graduating feeling confident in my skills. If you're finding it hard, know that it's completely normal a

In [11]:
# =============================================================================
# Try Your Own Query
# =============================================================================

your_query = "What's it like living in the dorms?"  # <-- Change this!

result = distributional_engine.query(your_query, return_metadata=True)
print(f"Q: {your_query}\n")
print(f"A: {result['response']}")
print()
print(f"[Based on: {result['sampled_confession'][:200]}...]")


Q: What's it like living in the dorms?

A: Living in the dorms can be pretty isolating, especially if you’re not actively seeking out connections. I felt like I missed out on the community vibe because I spent most of my time alone, focusing on classes and playing games. It’s a unique experience, but if you don’t make an effort to engage with others, it can feel lonely. I realized too late that getting involved in dorm events or hanging out in common spaces could have made a huge difference in my college experience.

[Based on: I spent most of my free time in my room alone playing league while high during pnr. I made a few friends but never really went out. I\u2019m taking like 60 units this semester, but I really don\u2019t...]


# Evaluation Setup

For clean evaluation against our OpinionQA-style annotated dataset, we need a separate index containing ONLY confessions from #70964 onwards. This ensures the model samples from the same distribution that the ground truth annotations are based on.


In [12]:
# =============================================================================
# Step 1: Filter confessions to #70964+ for evaluation
# =============================================================================

import json
import re
import os

# Configuration
MIN_CONFESSION_NUM = 70964
INPUT_FILE = "data/all_confessions_cleaned.json"
OUTPUT_DIR = "data_eval"
OUTPUT_FILE = f"{OUTPUT_DIR}/confessions_eval.json"

def extract_confession_number(text):
    """Extract confession number from text like '#70964: some text...'"""
    match = re.match(r'#(\d+)', text)
    return int(match.group(1)) if match else None

# Load all confessions
print(f"Loading confessions from {INPUT_FILE}...")
with open(INPUT_FILE, 'r') as f:
    all_confessions = json.load(f)
print(f"Total confessions loaded: {len(all_confessions)}")

# Filter to #70964+
eval_confessions = []
for confession in all_confessions:
    num = extract_confession_number(confession.get('text', ''))
    if num is not None and num >= MIN_CONFESSION_NUM:
        confession['confession_num'] = num  # Add as explicit field for convenience
        eval_confessions.append(confession)

print(f"Confessions >= #{MIN_CONFESSION_NUM}: {len(eval_confessions)}")

# Sort by confession number (oldest first for consistency)
eval_confessions.sort(key=lambda x: x['confession_num'])

print(f"Confession range: #{eval_confessions[0]['confession_num']} to #{eval_confessions[-1]['confession_num']}")

# Save filtered dataset
os.makedirs(OUTPUT_DIR, exist_ok=True)
with open(OUTPUT_FILE, 'w') as f:
    json.dump(eval_confessions, f, indent=2)
print(f"✓ Saved {len(eval_confessions)} confessions to {OUTPUT_FILE}")


Loading confessions from data/all_confessions_cleaned.json...
Total confessions loaded: 65225
Confessions >= #70964: 5598
Confession range: #70964 to #661493
✓ Saved 5598 confessions to data_eval/confessions_eval.json


In [13]:
# =============================================================================
# Step 2: Create evaluation index from filtered confessions
# =============================================================================
# Same pattern as the original index creation in cell 0

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

EVAL_STORAGE_DIR = "./storage_eval"

# Check if eval index already exists
if os.path.exists(EVAL_STORAGE_DIR) and os.listdir(EVAL_STORAGE_DIR):
    print(f"Loading existing eval index from {EVAL_STORAGE_DIR}...")
    storage_context = StorageContext.from_defaults(persist_dir=EVAL_STORAGE_DIR)
    eval_index = load_index_from_storage(storage_context)
    print("✓ Loaded existing eval index")
else:
    print(f"Creating new eval index (this will call OpenAI embeddings API)...")
    
    # Same pattern as original: SimpleDirectoryReader → VectorStoreIndex
    documents = SimpleDirectoryReader(OUTPUT_DIR).load_data()
    eval_index = VectorStoreIndex.from_documents(documents)
    
    # Persist for reuse
    os.makedirs(EVAL_STORAGE_DIR, exist_ok=True)
    eval_index.storage_context.persist(persist_dir=EVAL_STORAGE_DIR)
    print(f"✓ Eval index created and saved to {EVAL_STORAGE_DIR}")


2025-12-04 01:22:18,974 - WARNING - `llama-index-readers-file` package not found, some file readers will not be available if not provided by the `file_extractor` parameter.


Creating new eval index (this will call OpenAI embeddings API)...


2025-12-04 01:22:25,327 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-12-04 01:22:27,138 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-12-04 01:22:28,162 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-12-04 01:22:29,374 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-12-04 01:22:31,167 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-12-04 01:22:31,995 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-12-04 01:22:33,088 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-12-04 01:22:34,113 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-12-04 01:22:34,955 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-12-04 01:22:35,756 - INFO - HTTP

✓ Eval index created and saved to ./storage_eval


In [14]:
# =============================================================================
# Step 3: Create evaluation-ready Distributional Query Engine
# =============================================================================

# Use the eval_index (only #70964+ confessions)
eval_distributional_engine = DistributionalQueryEngine(eval_index, top_k=30)

print("✓ Evaluation Distributional Query Engine initialized")
print(f"  - Index: eval_index (confessions #{MIN_CONFESSION_NUM}+)")
print(f"  - Retrieval width: top_k={eval_distributional_engine.top_k}")
print(f"  - LLM: {eval_distributional_engine.llm.model}")


✓ Evaluation Distributional Query Engine initialized
  - Index: eval_index (confessions #70964+)
  - Retrieval width: top_k=30
  - LLM: gpt-4o-mini


In [18]:
# =============================================================================
# Step 4: Test the eval engine (quick sanity check)
# =============================================================================

test_query = "Should students be able to use AI assistance in their assignments?"

result = eval_distributional_engine.query(test_query, return_metadata=True)
print(f"Query: {test_query}\n")
print(f"Response: {result['response']}")
print()
print(f"[Retrieved {result['total_retrieved']} chunks from eval corpus]")
print(f"[Sampled chunk preview: {result['sampled_confession'][:200]}...]")


2025-12-04 01:25:51,279 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-12-04 01:25:53,536 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Query: Should students be able to use AI assistance in their assignments?

Response: Absolutely, I think students should be allowed to use AI assistance in their assignments. With the increasing capabilities of LLMs, it's becoming harder to compete without them, and I believe incorporating AI can actually enhance our learning experience rather than diminish it. Instead of punishing those of us who are trying to follow the rules, why not expand project expectations to include open discussions about AI usage? After all, I’m here to build a portfolio and develop skills, not just check boxes on my way to a job!

[Retrieved 30 chunks from eval corpus]
[Sampled chunk preview: And people have less bandwidth for their work while their psets and projects are compared to LLM outputs? I know cheaters are shooting themselves in the foot long-term but it\u2019s not like GPA is me...]


# Evaluation Harness

Run N=50 samples per query to generate output distributions that can be compared against OpinionQA ground truth.


In [20]:
# =============================================================================
# Evaluation: Run N samples and collect distribution
# =============================================================================
# This uses the DistributionalQueryEngine which queries the LlamaIndex vector store,
# randomly samples one result, and generates a response.

from tqdm import tqdm

def run_evaluation_samples(engine, query: str, n_samples: int = 50):
    """
    Run a query N times using the distributional engine.
    
    The engine uses LlamaIndex under the hood:
    - VectorIndexRetriever queries the vector store (top_k=30)
    - Randomly samples 1 chunk from results
    - LLM generates response embodying that perspective
    
    Returns:
        dict with:
        - 'query': the input query
        - 'n_samples': number of samples taken  
        - 'responses': list of generated responses
        - 'sampled_chunks': list of source chunks that were sampled
    """
    responses = []
    sampled_chunks = []
    
    print(f"Running {n_samples} samples for query: '{query}'")
    for _ in tqdm(range(n_samples)):
        result = engine.query(query, return_metadata=True)
        responses.append(result['response'])
        sampled_chunks.append(result['sampled_confession'])
    
    return {
        'query': query,
        'n_samples': n_samples,
        'responses': responses,
        'sampled_chunks': sampled_chunks
    }

def print_evaluation_summary(eval_result):
    """Print a summary of the evaluation results."""
    print(f"\nQuery: '{eval_result['query']}'")
    print(f"Total samples: {eval_result['n_samples']}")
    print(f"\nSample responses:")
    for i, resp in enumerate(eval_result['responses'][:3], 1):
        print(f"\n--- Response {i} ---")
        print(resp[:300] + "..." if len(resp) > 300 else resp)

print("✓ Evaluation functions defined")


✓ Evaluation functions defined


In [ ]:
# =============================================================================
# Run Evaluation (50 samples)
# =============================================================================

EVAL_QUERY = "Should students be able to use AI assistance in their assignments?"
N_SAMPLES = 10

eval_results = run_evaluation_samples(eval_distributional_engine, EVAL_QUERY, n_samples=N_SAMPLES)
print_evaluation_summary(eval_results)

# To save results for later analysis:
import pickle
with open('eval_results.pkl', 'wb') as f:
    pickle.dump(eval_results, f)

# print("Ready to run evaluation. Uncomment the lines above when ready.")


Running 10 samples for query: 'Should students be able to use AI assistance in their assignments?'


  0%|          | 0/10 [00:00<?, ?it/s]2025-12-04 01:29:23,261 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-12-04 01:29:25,703 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
 10%|█         | 1/10 [00:02<00:23,  2.60s/it]2025-12-04 01:29:25,824 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-12-04 01:29:28,167 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
 20%|██        | 2/10 [00:05<00:20,  2.52s/it]2025-12-04 01:29:28,354 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-12-04 01:29:30,319 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
 30%|███       | 3/10 [00:07<00:16,  2.35s/it]2025-12-04 01:29:30,446 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-12-04 01:29:32,879 - INFO - HTTP Request: POST https


Query: 'Should students be able to use AI assistance in their assignments?'
Total samples: 10

Sample responses:

--- Response 1 ---
Absolutely, I believe students should be able to use AI assistance in their assignments. As someone who has faced the financial pressures of education, I see AI as a tool that can level the playing field, helping us grasp complex concepts and enhance our learning. It's not about taking shortcuts; it...

--- Response 2 ---
Absolutely, students should be able to use AI assistance in their assignments. We're already dealing with so much stress from the grading system and the pressure to perform well, so leveraging AI can help us enhance our understanding and creativity without adding to the anxiety. It’s not about takin...

--- Response 3 ---
Absolutely, students should be able to use AI assistance in their assignments. Just like any tool, AI can enhance our learning and help us tackle complex problems more efficiently. It’s about leveraging technology to u

In [23]:
# =============================================================================
# Diagnostic: Inspect what chunks are actually being retrieved
# =============================================================================

test_query = "Should students be able to use AI assistance in their assignments?"

# Get raw retrieval results (before random sampling)
retrieved_nodes = eval_distributional_engine.retriever.retrieve(test_query)

print(f"Query: '{test_query}'")
print(f"Retrieved {len(retrieved_nodes)} chunks\n")

# Show first 5 chunks
for i, node in enumerate(retrieved_nodes[:5]):
    print(f"{'='*70}")
    print(f"CHUNK {i+1} (similarity: {node.score:.3f})")
    print(f"{'='*70}")
    # Show first 500 chars of the chunk
    content = node.get_content()
    print(content[:500])
    print("..." if len(content) > 500 else "")
    print()


2025-12-04 14:45:13,484 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Query: 'Should students be able to use AI assistance in their assignments?'
Retrieved 30 chunks

CHUNK 1 (similarity: 0.803)
Still, my parents didn\u2019t speak English or have a lick of knowledge about the American education system or the fields I was interested in. I wanted to offer my experiences to balance out the doom and gloom floating around.\n\nI\u2019m an Olympiad kid majoring in something unrelated to my Olympiad. But I know many non-Olympiad kids who are having a jolly time here as well, and these have truly been both the best and the least stressful years of my life. (Idk maybe that only speaks to how bad 
...

CHUNK 2 (similarity: 0.787)
I was tutoring someone for 1802 and reading through  one of their  solutions when I happen to stumble upon what I feel as a clever method that was not in the solution set for the problems I was using. It turned out that the student had a large misunderstanding. The expression on my face changed which prompted the student to ask what happen